In [1]:
# !pip install pinecone-client

  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [4]:
from pinecone import Pinecone, ServerlessSpec
import os

pinecone_api = os.environ["PINECONE_API_KEY"]

# vectorstore load
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = "ragtest"

# if not have that index
pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=spec,
)

In [5]:
index = pc.Index(index_name)


In [6]:
# !pip install langchain-pinecone

  Using cached aiohttp-3.9.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl.metadata (2.2 kB)
Using cached pinecone_plugin_inference-3.1.0-py3-none-any.whl (87 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inference-1.1.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 3.1.0 which is incompatible.


In [7]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


In [8]:

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(embedding=embeddings, index=index)
vector_store.add_documents(documents=page_all)

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv


load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
  os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]

# vectorstore load
pc = Pinecone(api_key=pinecone_api)
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

index_name = "ragtest"

# if not have that index
pc.create_index(
    name=index_name,
    dimention=1536,
    metric="cosine",
    spec=spec,
)

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)


from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-3-large")

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

print(index.describe_index_stats())
